# TIES protocol with BAC2.0

In this tutorial we will implement the NAMD version of the TIES protocol. We will start with the **minimization** steps.

In [1]:
from bac.simulate.namd import (Simulation, NonBondedController, PME, BondConstraint, AtomConstraint, 
                               HarmonicConstraint, non_bonded_controller, VelocityRescaling, VelocityReassignment,
                               LangevinDynamics, BerendsenPressureCoupling, FreeEnergyController)

In [2]:
minimize = Simulation(timestep=2.0, number_of_steps=1000, temperature=300)
minimize.minimization = True

minimize.parameter_type_CHARMM = False
minimize.amber = True
minimize.parameters = 'build/complex.top'
minimize.coordinates = 'build/complex.pdb'
minimize.read_exclusions = False

minimize.binary_output = False
minimize.name = 'minimize_0'
minimize.output_name = 'minimize_0'
minimize.output_energies = 500
minimize.output_pressure = 500

Next, the non bonded interaction setting.

In [2]:
nbc = NonBondedController()
nbc.excluded_interactions = non_bonded_controller.Interaction.scaled_one_four
nbc.one_four_scaling = 0.833333
nbc.cutoff = 12
nbc.switching = True
nbc.switching_distance = 10
nbc.pairlist_distance = 13.5

nbc.non_bonded_frequency = 1
nbc.full_elect_frequency = 2
nbc.steps_per_cycle = 10

nbc.pme = PME(grid_spacing=1.0)

nbc.wrap_water, nbc.wrap_all = True, True
nbc.cell_basis_vector_1 = 87.425, 0.000, 0.000
nbc.cell_basis_vector_2 =  0.000,97.836, 0.000
nbc.cell_basis_vector_3 =  0.000, 0.000,76.401
nbc.cell_origin =         -0.093,-0.273,-0.306

# minimize.non_bonded_controller = nbc

TypeError: isinstance() arg 2 must be a type or tuple of types

In [4]:
minimize.constraints.bond_constraint = BondConstraint(bonds='all', tolerance=0.00001, iterations=100)

harm_const = HarmonicConstraint(exponent=2, reference_position_file='build/complex.pdb')
harm_const.force_constant_file = 'constraint/f4.pdb'
harm_const.force_constant_column = 'B'

minimize.constraints.harmonic_constraint = harm_const

In [5]:
minimize.temperature_controller = LangevinDynamics(temperature=300, damping=5, applies_to_hydrogen=False)

Most importantly the free energy calculation parameters:

In [6]:
fe = FreeEnergyController(type='ti', file='build/tags.pdb', column='O', decouple=True)

fe.output_name = 'min.alch'
fe.output_frequency = 1000
fe.van_der_waals_shift_coefficient = 5
fe.electronic_interaction_start_lambda = 0.45
fe.van_der_waals_end_lambda = 1.0

minimize.free_energy_controller = fe


Now the `md` object is ready to run. We need to add it onto an operation queue to actually run in on a supercomputer.

Under the hood, BAC creates an `.conf` file that the NAMD binary knows how to run. You can also have a look at this file by simply encoding the `Run` object. This is how it's done:

# Equilibration step 1

In [8]:
eq1 = next(minimize)

In [15]:
eq1.name = 'eq1'
eq1.output_name = 'eq1'
eq1.minimization = False

eq1.number_of_steps = 30000

eq1.restart_frequency = 50000
eq1.non_bonded_controller.xst_frequency = 5000
eq1.dcd_frequency = 5000

# Equilibration step 2

In [22]:
eq2 = next(eq1)
eq2.name = 'eq2'
eq2.output_name = 'eq2'
eq2.number_of_steps = 970_000

eq2.pressure_controller = BerendsenPressureCoupling(target=1.0, compressibility=4.57e-5, 
                                                    relaxation_time=100, frequency=2, use_group_pressure=True)


/Users/kristofarkas/Developer/BAC2/bac/utils/decorators.py:99: UserWarning: 'BerendsenPressureCoupling' object has no attribute 'simulation'
  warnings.warn(f'{e}')


# Production

In [26]:
prod = next(eq2)
prod.name = 'prod'
prod.output_name = 'prod'
prod.constraints.harmonic_constraint = None
prod.restart_frequency = 200_000
prod.number_of_steps = 2_000_000

In [27]:
print(prod.encode())

timestep                  = 2.0
firsttimestep             = 0
minimization              = off
velocities                = eq2.vel
coordinates               = eq2.coor
parmfile                  = build/complex.top
outputname                = prod
binaryoutput              = no
restartfreq               = 200000
dcdfreq                   = 5000
dcdunitcell               = yes
outputEnergies            = 500
outputPressure            = 500
amber                     = yes
readexclusions            = no
scnb                      = 2
paraTypeXplor             = on
paraTypeCharmm            = off
gromacs                   = off
cutoff                    = 12.0
switching                 = on
switchdist                = 10.0
vdwForceSwitching         = off
exclude                   = scaled1-4
1-4scaling                = 0.833333
dielectric                = 1.0
nonbondedScaling          = 1.0
vdwGeometricSigma         = False
limitdist                 = 0.0
LJcorrection              = False
non